In [13]:
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import io

In [2]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

In [3]:
# 25000 samples for training and testing
train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_sequences = []
training_labels = []

testing_sequences = []
testing_labels = []

# loop over all train samples and save the sentences and labels 
for s, l in train_data: 
    training_sequences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

for s, l in test_data: 
    testing_sequences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

A review is stored in tensor as 
tf.Tensor(b'this is the review for a movie', shape=(), dtype=string)

While a labels is stored as 
tf.Tensor(1, shape=(), dtype=int64)

In [5]:
# convert the labels to numpy arrays
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sequences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sequences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sequences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 6)                 102       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(padded, training_labels_final, epochs=10, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 39s 43ms/step - loss: 0.5495 - accuracy: 0.7545 - val_loss: 0.3894 - val_accuracy: 0.8330
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3322 - accuracy: 0.8633 - val_loss: 0.3309 - val_accuracy: 0.8583
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2711 - accuracy: 0.8893 - val_loss: 0.3279 - val_accuracy: 0.8582
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2362 - accuracy: 0.9081 - val_loss: 0.3414 - val_accuracy: 0.8541
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2090 - accuracy: 0.9216 - val_loss: 0.3610 - val_accuracy: 0.8500
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1885 - accuracy: 0.9309 - val_loss: 0.3868 - val_accuracy: 0.8424
Epoch 7/10
782/782 [==============================] - 7s 8ms/step - loss: 0.1708 - accuracy: 0.9387 - val_loss: 0.4147 - val_accuracy: 0.8394


In [11]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [12]:
# reverse word_index to get words instead of keys
reverse_word_index = tokenizer.index_word

In [14]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + '\n')
  out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')

out_v.close()
out_m.close()